WUM - Zadanie 2
Stanisław Bitner 438247

In [ ]:
import os

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from scipy.sparse import csr_matrix
from scipy.cluster.hierarchy import linkage, leaves_list

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

from tabulate import tabulate

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l1, l2

In [ ]:
X_train_original = pd.read_csv('X_train.csv', sep=',')
X_test_original = pd.read_csv('X_test.csv', sep=',')
y_train_original = pd.read_csv('y_train.csv', sep=',')

In [ ]:
X_train = X_train_original
X_test = X_test_original

y_train = y_train_original.drop(columns=['Id'])
# Upewnij się, że y_train jest serią (Series), a nie DataFrame
y_train = y_train.squeeze()

In [ ]:
# Parametry

n_jobs = 8
n_splits = 5 # Liczba podziałów w walidacji krzyżowej - standardowe 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=438247)

# 1. Eksploracja (7 pkt.)
## (a) 
Sprawdź, ile obserwacji i zmiennych zawierają wczytane dane treningowe oraz testowe. Przyj-
rzyj się typom zmiennych i, jeśli uznasz to za słuszne, dokonaj odpowiedniej konwersji przed
dalszą analizą. Upewnij się, czy dane są kompletne.
## (b) 
Zbadaj rozkład empiryczny zmiennej objaśnianej (przedstaw kilka podstawowych statystyk,
do analizy dołącz histogram lub wykres estymatora gęstości).
## (c) 
Wybierz 250 zmiennych objaśniających najbardziej skorelowanych ze zmienną objaśnianą. Po-
licz korelację dla każdej z par tych zmiennych. Zilustruj wynik za pomocą mapy ciepła (heat-
map).

Uwaga: opisany tu wybór zmiennych jest tylko na potrzeby niniejszego podpunktu, analizę
opisaną w kolejnych zadaniach należy przeprowadzić na pełnym zbiorze danych treningowych.

In [ ]:
# Sprawdź liczbę obserwacji i zmiennych
print("Liczba obserwacji i zmiennych w X_train:")
print(X_train.shape)
print("Liczba obserwacji i zmiennych w X_test:")
print(X_test.shape)
print("Liczba obserwacji i zmiennych w y_train:")
print(y_train.shape)

In [ ]:
# Przyjrzyj się typom zmiennych
print("\nTypy zmiennych w X_train:")
print(X_train.dtypes)
print("\nTypy zmiennych w X_test:")
print(X_test.dtypes)
print("\nTypy zmiennych w y_train:")
print(y_train.dtypes)

In [ ]:
# Sprawdź kompletność danych
print("\nLiczba brakujących danych w X_train:")
print(X_train.isnull().sum())
print("\nLiczba brakujących danych w X_test:")
print(X_test.isnull().sum())
print("\nLiczba brakujących danych w y_train:")
print(y_train.isnull().sum())

Wszystkie dane są typu float i są kompletne.

In [ ]:
# Oblicz podstawowe statystyki opisowe
print("Podstawowe statystyki opisowe zmiennej objaśnianej (y_train):")
print(y_train.describe())

In [ ]:
# Tworzenie histogramu
plt.figure(figsize=(10, 6))
sns.histplot(y_train, kde=False, bins=30)
plt.title('Histogram zmiennej objaśnianej (y_train)')
plt.xlabel('Wartość')
plt.ylabel('Częstość')
plt.show()

Wykres pokazuje, że dane tworzą wykres zbliżony do normalnego z bardzo dużym dodatkiem zer.
To oznacza, że można spodziewać się wielu zupełnie niepoprawnych predykcji -- model może na przykład przewidywać zawsze 0 i wciąż będzie miał całkiem dobrą dokładność, co oczywiście mija się z celem.

In [ ]:
# Tworzenie wykresu estymatora gęstości
plt.figure(figsize=(10, 6))
sns.kdeplot(y_train.squeeze(), fill=True)
plt.title('Estymator gęstości zmiennej objaśnianej (y_train)')
plt.xlabel('Wartość')
plt.ylabel('Gęstość')
plt.show()

In [ ]:
correlations = X_train.corrwith(y_train, method='spearman').abs()
top_features = correlations.sort_values(ascending=False).head(250).index
X_top = X_train[top_features]
corr_matrix = X_top.corr(method='spearman')
plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Mapa ciepła korelacji pomiędzy 250 najbardziej skorelowanymi zmiennymi objaśniającymi')
plt.show()

Mapa ciepła dla korelacji Spearmana. Niestety niewiele można o niej powiedzieć, ze względu na dużą liczbę zmiennych.
Wybrałem korelację Spearmana, gdyż korelacja Pearsona zakłada liniowość, której tu nie ma.

# 2. ElasticNet (7 pkt.)
Pierwszy model, który należy wytrenować, to ElasticNet. Którego szczególne przypadki stanowią
regresja grzbietowa (ridge regression) oraz lasso.
## (a) 
Przedstaw w raporcie informacje o modelu ElasticNet, objaśniając parametry, które są w nim
estymowane, optymalizowaną funkcję oraz hiperparametry, od których ona zależy. Dla jakich
wartości hiperparametrów otrzymujemy regresję grzbietową, a dla jakich lasso?
## (b) 
Zdefiniuj siatkę (grid ) hiperparametrów, opartą na co najmniej trzech wartościach każdego
z hiperparametrów. Zadbaj o to, by w siatce znalazły się konfiguracje hiperparametrów od-
powiadające regresji grzbietowej i lasso. Użyj walidacji krzyżowej do wybrania odpowiednich
hiperparametrów (o liczbie podzbiorów użytych w walidacji krzyżowej należy zdecydować sa-
modzielnie oraz uzasadnić swój wybór).
## (c) 
Podaj błąd treningowy i walidacyjny modelu (należy uśrednić wynik względem wszystkich
podzbiorów wyróżnionych w walidacji krzyżowej).

### Opis

Wszystkie informacje pochodzą z tej strony - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

1. Opis Modelu ElasticNet

ElasticNet to liniowy model regresyjny, który łączy właściwości regresji grzbietowej (Ridge Regression) i Lasso (Least Absolute Shrinkage and Selection Operator). Główna zaleta ElasticNet polega na zdolności do jednoczesnego radzenia sobie z problemem wielokrotnej współliniowości (jak w Ridge Regression) oraz wykonywania selekcji cech (jak w Lasso).

2. Parametry Estymowane przez ElasticNet

- **Współczynniki regresji** (`coefficients`): Wagi przypisane każdej zmiennej objaśniającej, które są estymowane podczas trenowania modelu. Te wagi są interpretowane jako wpływ każdej zmiennej objaśniającej na zmienną objaśnianą.

3. Optymalizowana Funkcja

ElasticNet minimalizuje funkcję straty, która jest kombinacją sumy kwadratów reszt (RSS - Residual Sum of Squares) oraz regularizacji L1 i L2:

$$
\text{Loss}(w, b) = \frac{1}{2n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \alpha \left( \rho \sum_{j=1}^{p} |w_j| + \frac{1 - \rho}{2} \sum_{j=1}^{p} w_j^2 \right)
$$

gdzie:
- $ y_i $ to rzeczywiste wartości.
- $ \hat{y}_i $ to przewidywane wartości.
- $ w $ to wektory współczynników regresji.
- $ b $ to wyraz wolny (intercept).
- $ n $ to liczba próbek.
- $ p $ to liczba zmiennych objaśniających.
- $ \alpha $ to parametr regularyzacji, który kontroluje siłę regularizacji.
- $ \rho $ to parametr balansujący między regularyzacją L1 a L2.

4. Hiperparametry

ElasticNet ma dwa główne hiperparametry:
- **Alpha ($\alpha$)**: Kontroluje siłę całkowitej regularizacji. Wysoka wartość \(\alpha\) powoduje silniejszą regularizację, co może prowadzić do większego zredukowania współczynników.
- **L1_ratio ($\rho$)**: Kontroluje mieszankę regularyzacji L1 (Lasso) i L2 (Ridge). \(\rho\) w zakresie od 0 do 1, gdzie:
  - $\rho = 1$: Model staje się Lasso, używając wyłącznie regularyzacji L1.
  - $\rho = 0$: Model staje się Ridge Regression, używając wyłącznie regularyzacji L2.
  - $0 < \rho < 1$: ElasticNet używa zarówno regularyzacji L1, jak i L2.


In [ ]:
%%time
# Definiowanie siatki hiperparametrów
param_grid = {
    'elasticnet__alpha': [0.1, 1.0, 10.0],
    'elasticnet__l1_ratio': [0.0, 0.5, 1.0]
}
# ElasticNet wymaga standaryzacji zmiennych
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet())
])
# Użycie walidacji krzyżowej
grid_search_en = GridSearchCV(
    estimator=pipeline,
    n_jobs=n_jobs,
    param_grid=param_grid,
    cv=kf,
    scoring='neg_root_mean_squared_error',
    return_train_score=True
)
grid_search_en.fit(X_train, y_train)

Jako wartości siatki parametrów wybrałem te najbardziej standardowe:
- $\alpha \in \{0, 1, 10\}$;
- $\rho \in \{0, 0.5, 1\}$.
 
Oczywiście dla skrajnych wartości hiperparametrów otrzymujemy modele Ridge oraz Lasso.

Walidację krzyżową przeprowadziłem 5-krotnie, co jest dobrym kompromisem między szybkością otrzymania danych, a także dokładnością wyniku.

Ridge oraz Lasso wymagają, aby dane były ustandaryzowane, więc użyłem Pipeline'a, aby robił to sam używając StandardScalera. Skaler nie jest on idealny, a dane po zastosowaniu go wciąż pozostawiają wiele do życzenia, ale po konsultacji z Prowadzącą Laboratoria, nie było lepszych pomysłów.

In [ ]:
# Najlepsze parametry i wynik
best_params = grid_search_en.best_params_
print("Najlepsze parametry: ", best_params)

# Obliczenie średniego błędu treningowego i walidacyjnego
cv_results_en = grid_search_en.cv_results_
mean_train_rmse_en = -cv_results_en['mean_train_score'].mean()
print("Średni błąd treningowy (RMSE): ", mean_train_rmse_en)
mean_val_rmse_en = -cv_results_en['mean_test_score'].mean()
print("Średni błąd walidacyjny (RMSE): ", mean_val_rmse_en)

In [ ]:
%%time
# Definiowanie siatki hiperparametrów
param_grid = {
    'model__alpha': [0.1, 1.0, 10.0],
}
# Ridge wymaga standaryzacji zmiennych
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge())
])
grid_search_ridge = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=kf,
    scoring='neg_root_mean_squared_error',
    n_jobs=n_jobs,
    return_train_score=True
)
grid_search_ridge.fit(X_train, y_train)

In [ ]:
# Najlepsze parametry i wynik
best_params = grid_search_ridge.best_params_
print("Najlepsze parametry: ", best_params)

# Obliczenie średniego błędu treningowego i walidacyjnego
cv_results_ridge = grid_search_ridge.cv_results_
mean_train_rmse_ridge = -cv_results_ridge['mean_train_score'].mean()
print("Średni błąd treningowy (RMSE): ", mean_train_rmse_ridge)
mean_val_rmse_ridge = -cv_results_ridge['mean_test_score'].mean()
print("Średni błąd walidacyjny (RMSE): ", mean_val_rmse_ridge)

Oprócz modelu ElasticNet, który ma problem z regresją grzbietową użyłem także modelu Ridge, który jest do niej stworzony i zoptymalizowany.
Użycie go było również wspomniane w ostrzeżeniach jakie się pojawiały podczas wykonywania GridSearchCV na modelu ElasticNet.

*Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.*

Sam ElasticNet okazał się faworyzować Redge względem Lassa i optymalnymi parametrami okazały się $\alpha=0.1, \rho=0$, co jest równoważne regresji grzbietowej dla $\alpha=0.1$. Sama regresja grzbietowa zachowywała się jednak najlepiej dla $\alpha=10$.

Ostatecznie oczywiście Ridge dał lepsze średnie RMSE ($0.44$) w porównaniu do ElasticNat-a ($0.69$), który do średniej wliczał również Lasso.

# 3. Lasy losowe (8 pkt.)
W tej części projektu należy wytrenować model lasów losowych i porównać jego działanie z utwo-
rzonym wcześniej modelem ElasticNet.
## (a) 
Spośród wielu hiperparametrów charakteryzujących model lasów losowych wybierz trzy róż-
ne. Zdefiniuj trójwymiarową siatkę przeszukiwanych kombinacji hiperparametrów i za pomocą
walidacji krzyżowej wybierz ich optymalne (w kontekście wykonywanej predykcji) wartości.
Wykorzystany przy walidacji krzyżowej podział danych powinien być taki sam, jak w przy-
padku ElasticNet.
## (b) 
Zrób podsumowanie tabelaryczne wyników, jakie otrzymywały metody w walidacji krzyżowej
w obu rozważanych modelach. (Porównanie to jest powodem, dla którego zależy nam na zasto-
sowaniu tych samych podziałów). Określ, który model wydaje Ci się najlepszy (uzasadnij swój
wybór). Do porównania dołącz podstawowy model referencyjny, który dowolnym wartościom
zmiennych objaśniających przypisuje średnią arytmetyczną zmiennej objaśnianej.

In [ ]:
%%time
# Definiowanie siatki hiperparametrów
param_grid = {
    'n_estimators': [10, 20, 50],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}
# Użycie walidacji krzyżowej z 5 podzbiorami (5-fold cross-validation)
grid_search_rf = GridSearchCV(
    estimator=RandomForestRegressor(n_jobs=n_jobs),
    n_jobs=n_jobs,
    param_grid=param_grid,
    cv=kf,
    scoring='neg_root_mean_squared_error',
    return_train_score=True
)
# Dane mają gęstość ~10%, więc można zrobić csr_matrix
grid_search_rf.fit(csr_matrix(X_train.values), y_train)

In [ ]:
# Najlepsze parametry i wynik
best_params = grid_search_rf.best_params_
print("Najlepsze parametry: ", best_params)

# Obliczenie średniego błędu treningowego i walidacyjnego
cv_results_rf = grid_search_rf.cv_results_
mean_val_rmse_rf = -cv_results_rf['mean_test_score'].mean()
print("Średni błąd walidacyjny (RMSE): ", mean_val_rmse_rf)

Przede wszystkim, aby zoptymalizować wykonywanie się modelu, korzystając z niskiej gęstości danych (~10%), przerobiłem dane na csr_matrix, co zmniejszyło czasy wykonania około 4-krotnie.

Wybrałem parametry *n_estimators* (liczba drzew w lesie), *max_depth* (maksymalna głębokość drzewa decyzyjnego)m *min_samples_split* (minimalna liczba próbek wymagana do podziału węzłów).

Parametry te są standardowym wyborem.
Wartości parametrów wybrałem takie, aby zachodził kompromis między biasem a wariancją modelu RandomForest oraz w celu zoptymalizowania jego dokładności i wydajności.

Większa liczba drzew nie przynosiła wystarczająco dobrych rezultatów w porównaniu z czasem dopasowywania modelu do danych.
Podobnie było z maksymalną głębokością drzew.

Podobnie jak w przypadku ElasticNet, kroswalidacje wykonałem 5-krotnie, aby modele były lepiej porównywalne.

In [ ]:
# Błąd referencyjnego modelu (średni błąd na zbiorze treningowym)
mean_val_rmse_ref = np.sqrt(np.mean((y_train - np.mean(y_train))**2))

# Tabela z wynikami
results_with_ref = [
    ["ElasticNet",            mean_val_rmse_en],
    ["Ridge",                 mean_val_rmse_ridge],
    ["RandomForestRegressor", mean_val_rmse_rf],
    ["Referencyjny model",    mean_val_rmse_ref]
]

# Tabela z wynikami z referencyjnym modelem
print(tabulate(
    results_with_ref,
    headers=["Model", "Średni błąd walidacyjny (RMSE)"],
    tablefmt="fancy_grid"
))

# TODO: opis

# 4. Predykcja na zbiorze testowym (8 pkt.)
Ta część projektu ma charakter otwarty. W oparciu o dane treningowe należy dopasować dowolnie
wybrany model, a następnie zastosować go do przewidywania wartości zmiennej objaśnianej w
zbiorze testowym. Sposób wyboru i budowy modelu, a także motywacje stojące za takim wyborem
powinny zostać opisane w raporcie. Wygenerowane predykcje należy wysłać do prowadzącego w
osobnym pliku, którego format został opisany wcześniej. Liczba uzyskanych punktów zależeć będzie
od jakości predykcji, mierzonej pierwiastkiem błędu średniokwadratowego, RMSE.
Szczegóły punktacji:
* (1 pkt.) – za błąd niższy od pochodzącego z opisanego wcześniej, podstawowego modelu referencyj-
nego.
* (2 pkt.) – za błąd niższy od pochodzącego z modelu ElasticNet wytrenowanego przez prowadzących
laboratoria.
* (5 pkt.) – ten bonus obliczany jest według wzoru 12 ⌊10Fb(e)3 ⌋, gdzie e to błąd testowy predykcji studenta, Fb jest dystrybuantą empiryczną błędów wszystkich zgłoszonych predykcji w grupie laboratoryjnej studenta, natomiast ⌊·⌋ to część całkowita.

Ze względu na charakter danych ich normalizacja, czy też transformacja logarytmiczna psują je.
To powoduje, że tak na prawdę zostajemy z dwiema możliwościami wyboru modelu: RandomForest i NeuralNetwork.

Wybrałem sieć neuronową.

Danych jest niecałem 4 tysiące, co jest dość niedużą liczbą. Najlepsze rezultaty sieci dawały dla od 512 do 4096 neuronów w każdej z ukrytych warstw. Liczba ukrytych warstw przy jakiej model zachowywał się sensownie i liczył w wystarczająco krótkim czasie, to od 3 do 5.

Standardowo wybierałem losowo 80% danych jako treningowe i 20% jako walidacyjne.

Z początku sprawdzałem 3 ukryte warstwy po 512 neuronów, każda z funkcją aktywacji ReLU.
Dawało to wyniki podobne do modelu RandomForest z zadania 3-go.
Aby wprowadzić nieliniowość zmieniłem funkcję aktywacji w środkowej warstwie na tanh, co przyniosło dobre efekty.

Ostatecznie sprawdziłem ~50 modeli używających różnych funkcji aktywacji, przy czym w każdej używałem ReLU i tanh.
Modele porównywałem na podstawie średniej wartości RMSE na walidacyjnych danych spośród 5 treningów.
Najlepszym okazał się model zawierający 5 warstw ukrytych (512, gelu), (384, tanh), (4096, sigmoid), (384, gelu), (256, relu).
Funkcja elu dawała bardzo duży rozrzut otrzymywanych błędów walidacyjnych, co oczywiście może skutkować słabymi wynikami na danych testowych, dlatego też ich nie używałem. Uznałem, że lepiej jest mieć model, który niezależnie od podziału danych daje podobne RMSE.
Co ciekawe wstawienie sigmoida z bardzo dużą liczbą neuronów okazało się mieć pozytywny efekt na rmse dawane przez model.

W rozwiązaniu sprawdzałem także zachowanie modelu, po dołożeniu warstw DropOut (około 0.3) i warstw BatchNormalization, jednak przyniosły one odwrotne do oczekiwanych skutki. Modele z tymi warstwami wykazywały tendencje, do znacznie gorszej predykcji, bez poprawy zjawisku overfittingu.

Podobnie nieefektywny okazał się być EarlyStopping.

Używanie regularyzacji l1 czy l2 nie przynosiło żadnych znaczących korzyści przy liczbie epok, na której trenowałem modele.

Jeśli chodzi o BatchSize, to dobrym kompromisem między szybkością trenowania, a dokładnością predykcji okazał się rozmiar 64.
Sprawdzałem również 16, 32 i 128, ale żaden z nich nie był zdecydowanie lepszy.

Jedynym trikiem, który dał pozytywne efekty było dostosowanie LearningRate (do $0.001$), co nieznacznie poprawiło błędy walidacji.

Liczba epok z jaką testowałem modele, to od 30 do 100, przy czym końcowy model ma ich 50.

In [ ]:
X_train = X_train_original
y_train = y_train_original.drop(columns=['Id'])
y_train = y_train.squeeze()

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
)

N = X_train.shape[1]
model = Sequential([
    Input(shape=(N,)),
    Dense(512, activation='gelu'),
    Dense(384, activation='tanh'),
    Dense(4096, activation='sigmoid'),
    Dense(384, activation='gelu'),
    Dense(256, activation='relu'),
    Dense(1,   activation='linear')
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)
history = model.fit(
    X_train_split, y_train_split,
    epochs=50,
    batch_size=64,
    validation_data=(X_val_split, y_val_split),
    verbose=0
)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

val_rmse = np.sqrt(mean_squared_error(y_val_split, model.predict(X_val_split)))

print("Błąd walidacyjny (RMSE): ", val_rmse)

In [ ]:
predictions = model.predict(X_test)
ids = np.arange(len(X_test))
df = pd.DataFrame({'Id': ids, 'Expected': predictions.flatten()})
df.to_csv('sb438247_predykcja.csv', index=False)